# Processing ACORD-127 forms using Textract post-processing libraries

## Use case : Hierarchical Key-Value mapping

When a document contains hierarchical structuring, it is an important IDP post-processing task to infer the context within the structures. For eg.: In the document sample of an ACORD 127 form - Page 3 below, we want to infer the relationships between each `Veh #` keys to their corresponding vehical information items within each highlighted section. This can be done using Textract post-processing libraries for hierarchical key-value pairs mapping. 


In [1]:
from PIL import Image
from IPython.display import Image, display, HTML, JSON, IFrame

documentName = "doc-samples/Acord-127-pg3-annotated.png"
display(IFrame(documentName, 500, 600));

## Step 1: Installation


### [Amazon Textract Geofinder](https://pypi.org/project/amazon-textract-geofinder/)
Amazon Textract package to easier access data through geometric information.

Provides functions to use geometric information to extract information.

<b>Use cases include:</b>

   -  Give context to key/value pairs from the Amazon Textract AnalyzeDocument API for FORMS
   -  Find values in specific areas
   
### Other helper libraries for Textract response parsing : 

- Using <b>call_textract( )</b> from the [Textract-Caller](https://github.com/aws-samples/amazon-textract-textractor/tree/c689441c0562afb4976d4f248559e59289a33777/caller) library makes it is easy to parse JSON responses from AnalyzeDocument API.

- Also using [Textract-PrettyPrinter](https://github.com/aws-samples/amazon-textract-textractor/tree/master/prettyprinter) library that provides functions to format the output received from Textract in more easily consumable formats such as CSV.

You will need to run the cell below only once for installation.


In [ ]:
!python -m pip install amazon-textract-helper amazon-textract-geofinder amazon-textract-caller amazon-textract-response-parser --upgrade 

In [ ]:
from textractgeofinder.ocrdb import AreaSelection
from textractgeofinder.tgeofinder import KeyValue, TGeoFinder, AreaSelection, SelectionElement
from textractprettyprinter.t_pretty_print import get_forms_string, convert_form_to_list_trp2

from textractcaller.t_call import call_textract
from textractcaller.t_call import Textract_Features

import trp.trp2 as t2

This is the image we want to extract information from.

In [ ]:
from PIL import Image
from IPython.display import Image, display, HTML, JSON, IFrame

documentName = "doc-samples/Acord-127-pg3.png"
display(IFrame(documentName, 500, 600));

Calling Amazon Textract with the textractcaller library is easy. 

In [ ]:
j = call_textract(input_document=documentName, features=[Textract_Features.FORMS, Textract_Features.TABLES])

In [ ]:
print(get_forms_string(j))

There are multiple blocktypes in JSON. We need to deserialize the JSON into blocks using trp library.

In [ ]:
#from textractcaller.t_call import call_textract, Textract_Features
from trp.trp2 import TDocument, TDocumentSchema
from trp.t_pipeline import order_blocks_by_geo
import trp

t_doc = TDocumentSchema().load(j)
# the ordered_doc has elements ordered by y-coordinate (top to bottom of page)
ordered_doc = order_blocks_by_geo(t_doc)
# send to trp for further processing logic
trp_doc = trp.Document(TDocumentSchema().dump(ordered_doc))
# print(trp_doc)


In [ ]:
for page in trp_doc.pages:
    for field in page.form.fields:
        key = field.key.text if field.key else ""
        value = field.value.text if field.value else ""
        print(key, ":" ,value)
        

In [ ]:
t_document = t2.TDocumentSchema().load(j)
doc_height = 1000
doc_width = 1000
geofinder_doc = TGeoFinder(j, doc_height=doc_height, doc_width=doc_width)

## Step 2 : Using BoundingBox information given by Textract

The [bounding box](https://docs.aws.amazon.com/textract/latest/dg/API_BoundingBox.html) around the detected page, text, key-value pair, table, table cell, or selection element on a document page. 

We will now find the `Top` variable which is the top coordinate of the bounding box as a ratio of overall document page height.

This top variable will be the Y coodinates for each of the `Veh #` from the ACORD 127 form.
The X and Y values that are returned are ratios of the overall document page size. For example, if the input document is 700 x 200 and the operation returns X=0.5 and Y=0.25, then the point is at the (350,50) pixel coordinate on the document page.
Since, our document dimensions is 1000x1000 (configured in the cell above), we multiple each `vehicle_top` variable by 1000 to get our Y coodinate for each vehicle number.

In [ ]:
vehicle_top_list = []
for page in trp_doc.pages:
    for field in page.form.fields:
        key = field.key.text if field.key else ""
        value = field.value.text if field.value else ""
        if (key).lower().startswith('veh'):
            vehicle_top = field.geometry.boundingBox.top
            vehicle_top_list.append((int(vehicle_top*1000)))

In [ ]:
vehicle_top_list

Define `set_hierachy_kv()` a helper function to add "virtual" keys which we use to indicate context.

In [ ]:
def set_hierarchy_kv(list_kv: list[KeyValue], t_document: t2.TDocument, page_block: t2.TBlock, prefix="BORROWER"):
    for x in list_kv:
        t_document.add_virtual_key_for_existing_key(key_name=f"{prefix}_{x.key.text}",
                                                    existing_key=t_document.get_block_by_id(x.key.id),
                                                    page_block=page_block)

Let's see what does the second coordinate for lower-right look like? We can simply run the geo-finder method `find_phrase_on_page()` to find the 4 places where <b>"Total prem:"</b> appears in the document sample. 

In [ ]:
geofinder_doc.find_phrase_on_page("total prem:", min_textdistance=0.99)

## Step 3 : Map the Vehicle # to their corresponding kv pairs

We find the relevant phrases in the document to specify the area of key value pairs related to the patient information.

We then use this information to add new key value pairs to the Amazon Textract Response JSON Schema

In [ ]:
# Vehicle 1 geo-info
top_left = t2.TPoint(y=vehicle_top_list[0], x=0)
total_prem_1 = geofinder_doc.find_phrase_on_page("total prem:", min_textdistance=0.99)[0]
lower_right = t2.TPoint(y=total_prem_1.ymin, x=doc_width)

# Vehicle 1 hierarchical key
form_fields = geofinder_doc.get_form_fields_in_area(
    area_selection=AreaSelection(top_left=top_left, lower_right=lower_right, page_number=1))
set_hierarchy_kv(list_kv=form_fields,
                 t_document=t_document,
                 prefix='Veh_1',
                 page_block=t_document.pages[0])

# Vehicle 2 geo-info
top_left = t2.TPoint(y=vehicle_top_list[1], x=0)
total_prem_2 = geofinder_doc.find_phrase_on_page("total prem:", min_textdistance=0.99)[1]
lower_right = t2.TPoint(y=total_prem_2.ymin, x=doc_width)

# Vehicle 2 hierarchical key
form_fields = geofinder_doc.get_form_fields_in_area(
    area_selection=AreaSelection(top_left=top_left, lower_right=lower_right, page_number=1))
set_hierarchy_kv(list_kv=form_fields,
                 t_document=t_document,
                 prefix='Veh_2',
                 page_block=t_document.pages[0])

# Vehicle 3 geo-info
top_left = t2.TPoint(y=vehicle_top_list[2], x=0)
total_prem_3 = geofinder_doc.find_phrase_on_page("total prem:", min_textdistance=0.99)[2]
lower_right = t2.TPoint(y=total_prem_3.ymin, x=doc_width)

# Vehicle 3 hierarchical key
form_fields = geofinder_doc.get_form_fields_in_area(
    area_selection=AreaSelection(top_left=top_left, lower_right=lower_right, page_number=1))
set_hierarchy_kv(list_kv=form_fields,
                 t_document=t_document,
                 prefix='Veh_3',
                 page_block=t_document.pages[0])

# Vehicle 4 geo-info
top_left = t2.TPoint(y=vehicle_top_list[3], x=0)
total_prem_4 = geofinder_doc.find_phrase_on_page("total prem:", min_textdistance=0.99)[3]
lower_right = t2.TPoint(y=total_prem_4.ymin, x=doc_width)

# Vehicle 4 hierarchical key
form_fields = geofinder_doc.get_form_fields_in_area(
    area_selection=AreaSelection(top_left=top_left, lower_right=lower_right, page_number=1))
set_hierarchy_kv(list_kv=form_fields,
                 t_document=t_document,
                 prefix='Veh_4',
                 page_block=t_document.pages[0])


In [ ]:
print(get_forms_string(t2.TDocumentSchema().dump(t_document)))

## Step 4 : Print forms in Pandas dataframe

In [ ]:
import pandas as pd
from textractprettyprinter.t_pretty_print import convert_form_to_list
from trp import Document

tdoc=Document(t2.TDocumentSchema().dump(t_document))

dfs = list()
for page in tdoc.pages:
    dfs.append(pd.DataFrame(convert_form_to_list(trp_form=page.form)))


In [ ]:
dfs[0]